In [1]:
import pandas as pd
import json, os, csv
import re

In [2]:
# Specify data directory

data_dir = os.path.join(os.path.dirname(os.getcwd()),'data')
data_dir

'Z:\\Jupyter\\Netflix_Movie_Filter\\data'

In [3]:
imdb_file_path = os.path.join(data_dir, "movie_list.json")
amazon_file_path = os.path.join(data_dir, "baseline_data_ful_info.csv")

In [4]:
%%time
df_imdb = pd.read_json(imdb_file_path)
df_amazon = pd.read_csv(amazon_file_path)

Wall time: 4.58 s


In [5]:
print(df_imdb.shape)
print(df_amazon.shape)

(65406, 17)
(14658, 16)


In [6]:
df_imdb.head()

actors  \
0     [Kevin Costner, Kelly Preston, John C. Reilly,...   
1     [Kurt Russell, Val Kilmer, Sam Elliott, Bill P...   
10    [Saoirse Ronan, Laurie Metcalf, Tracy Letts, L...   
100   [Ethan Hawke, Uma Thurman, Gore Vidal, Xander ...   
1000  [Gabriel Bateman, Darby Camp, Kiele Sanchez, G...   

                        countries  \
0                           [USA]   
1                           [USA]   
10                          [USA]   
100                         [USA]   
1000  [United Arab Emirates, USA]   

                                            description  \
0     After 19 years of playing the game he's loved ...   
1     A successful lawman's plans to retire anonymou...   
10    In 2002, an artistically inclined seventeen-ye...   
100   A genetically inferior man assumes the identit...   
1000  Two school kids strike up a friendship with an...   

                              directors  \
0                           [Sam Raimi]   
1     [George P. Cosmatos, Kevin Jarre]   
10                       [Greta Gerwig]   
100                     [Andrew Niccol]   
1000                     [Brandon Camp]   

                                             genre  \
0                          [Drama, Romance, Sport]   
1     [Action, Biography, Drama, History, Western]   
10                                 [Comedy, Drama]   
100                      [Drama, Sci-Fi, Thriller]   
1000                        [Crime, Drama, Family]   

                                   imdb_url  \
0     https://www.imdb.com/title/tt0126916/   
1     https://www.imdb.com/title/tt0108358/   
10    https://www.imdb.com/title/tt4925292/   
100   https://www.imdb.com/title/tt0119177/   
1000  https://www.imdb.com/title/tt1799516/   

                                                img_url             languages  \
0     https://m.media-amazon.com/images/M/MV5BZDgzY2...             [English]   
1     https://m.media-amazon.com/images/M/MV5BODRkYz...      [English, Latin]   
10    https://m.media-amazon.com/images/M/MV5BODhkZG...    [English, Spanish]   
100   https://m.media-amazon.com/images/M/MV5BNDQxOT...  [English, Esperanto]   
1000  https://m.media-amazon.com/images/M/MV5BMTc5Mz...             [English]   

      metascore rating  runtime  \
0          43.0  PG-13  137 min   
1          50.0      R  130 min   
10         94.0      R   94 min   
100        64.0  PG-13  106 min   
1000       53.0  TV-PG   87 min   

                                                tagline                 title  \
0     Billy Chapel must choose between the woman he ...  For Love of the Game   
1                                 I'm your huckleberry!             Tombstone   
10                                         Time to Fly.             Lady Bird   
100   How do you hide when you're running from yours...               Gattaca   
1000                                  A Hero Comes Home                 Benji   

      users_rating    votes    year  votes_int  
0              6.6   31,110  1999.0    31110.0  
1              7.8  122,341  1993.0   122341.0  
10             7.4  225,508  2017.0   225508.0  
100            7.8  274,534  1997.0   274534.0  
1000           6.3    3,518  2018.0     3518.0

In [7]:
print(len(df_imdb.title.unique()))

57208


In [8]:
df_amazon.head()

overall  verified   reviewTime      reviewerID        asin  \
0      5.0      True  04 13, 2015  A2OI3HU0KN4118  0005119367   
1      5.0      True  04 15, 2014  A385J7MEM932DL  0005119367   
2      5.0      True  04 27, 2017   AIVTZDMB297BG  0767001311   
3      4.0      True   12 2, 2016   AXOX48S8K994V  0767001311   
4      4.0      True  10 13, 2016  A3J9LLWHZH15HG  0767001311   

                 style     reviewerName  \
0  {'Format:': ' DVD'}       Shantal N.   
1  {'Format:': ' DVD'}              Jim   
2  {'Format:': ' DVD'}      Alan Denman   
3  {'Format:': ' DVD'}  Amazon Customer   
4  {'Format:': ' DVD'}               JJ   

                                          reviewText  \
0                             Thanks for everything.   
1  This one was really interesting!  Loved it com...   
2                                              funny   
3                                       Great series   
4  I happen to like Bob Newhart even though he is...   

                      summary  unixReviewTime vote image  \
0                  Five Stars      1428883200  NaN   NaN   
1                 Bible Story      1397520000  NaN   NaN   
2                  Five Stars      1493251200  NaN   NaN   
3                  Four Stars      1480636800  NaN   NaN   
4  Well done, amusing series.      1476316800  NaN   NaN   

                                         title  char_count  \
0                                   Joseph VHS        22.0   
1                                   Joseph VHS       208.0   
2  The Very Best of the Bob Newhart Show 6 VHS         5.0   
3  The Very Best of the Bob Newhart Show 6 VHS        12.0   
4  The Very Best of the Bob Newhart Show 6 VHS       535.0   

   reviewer_rating_count  title_rating_count  
0                     78                  50  
1                     82                  50  
2                    101                  42  
3                     64                  42  
4                     93                  42

In [9]:
len(df_amazon.title.unique())

1284

In [10]:
def preprocessing(text):
    """
    Preprocess titles 
    """
    # Remove VHS tags
    text = text.replace(" VHS", "")
    # Lower case
    text = text.lower()
    # Remove punctuations and numerics
    text = ''.join(char for char in text if char.isalnum())
    return text  

In [11]:
%%time
df_amazon["title_normalize"] = df_amazon.title.apply(preprocessing)
df_imdb["title_normalize"] = df_imdb.title.apply(preprocessing)

Wall time: 5.55 s


In [12]:
df_merge = df_amazon.merge(df_imdb, how = "inner", on = "title_normalize")

In [13]:
print(df_merge.shape)

(9525, 34)


In [14]:
df_merge.columns

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image', 'title_x', 'char_count', 'reviewer_rating_count',
       'title_rating_count', 'title_normalize', 'actors', 'countries',
       'description', 'directors', 'genre', 'imdb_url', 'img_url', 'languages',
       'metascore', 'rating', 'runtime', 'tagline', 'title_y', 'users_rating',
       'votes', 'year', 'votes_int'],
      dtype='object')

In [15]:
df_merge.head()

overall  verified   reviewTime      reviewerID        asin  \
0      4.0      True  09 14, 2017  A1BT9J2I6DC246  0767023765   
1      5.0      True  10 27, 2013  A2K53OYO4JJO28  0767023765   
2      5.0      True  09 12, 2013  A2TY359PR45ATP  0767023765   
3      5.0      True  05 21, 2013  A192UPT6KST3E2  0767023765   
4      5.0      True  04 10, 2009  A2XRMQA6PJ5ZJ8  0767023765   

                 style            reviewerName  \
0  {'Format:': ' DVD'}                  Debbie   
1  {'Format:': ' DVD'}         Pete N. Austira   
2  {'Format:': ' DVD'}  R. Edward Merrell, Jr.   
3  {'Format:': ' DVD'}          Gary W. Phelps   
4  {'Format:': ' DVD'}     Roger J. Buffington   

                                          reviewText  \
0                                               good   
1  Every has visons of Christmas morning - waking...   
2  I've read several histories of Washington and/...   
3  This is an incredible and accurate portrayal o...   
4  As our schools manage to do a poorer and poore...   

                                       summary  unixReviewTime  ...  \
0                                   Four Stars      1505347200  ...   
1                      A New Look at Christmas      1382832000  ...   
2                        Good History Lesson!!      1378944000  ...   
3                                 The Crossing      1369094400  ...   
4  Very well done -- splendid historical drama      1239321600  ...   

   languages metascore rating  runtime  \
0  [English]       NaN   None   89 min   
1  [English]       NaN   None   89 min   
2  [English]       NaN   None   89 min   
3  [English]       NaN   None   89 min   
4  [English]       NaN   None   89 min   

                                            tagline       title_y  \
0  Washington needed a victory, they got a miracle.  The Crossing   
1  Washington needed a victory, they got a miracle.  The Crossing   
2  Washington needed a victory, they got a miracle.  The Crossing   
3  Washington needed a victory, they got a miracle.  The Crossing   
4  Washington needed a victory, they got a miracle.  The Crossing   

  users_rating  votes    year votes_int  
0          7.1  1,763  2000.0    1763.0  
1          7.1  1,763  2000.0    1763.0  
2          7.1  1,763  2000.0    1763.0  
3          7.1  1,763  2000.0    1763.0  
4          7.1  1,763  2000.0    1763.0  

[5 rows x 34 columns]

In [16]:
len(df_merge.title_normalize.unique())

524

In [18]:
df_merge.columns

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image', 'title_x', 'char_count', 'reviewer_rating_count',
       'title_rating_count', 'title_normalize', 'actors', 'countries',
       'description', 'directors', 'genre', 'imdb_url', 'img_url', 'languages',
       'metascore', 'rating', 'runtime', 'tagline', 'title_y', 'users_rating',
       'votes', 'year', 'votes_int'],
      dtype='object')

In [21]:
df_merge.shape

(9525, 34)

In [17]:
df_merge.title_normalize.unique()[0:524]

array(['thecrossing', 'steelmagnolias', 'easyrider',
       'closeencountersofthethirdkind', 'frightnight', 'annie',
       'doloresclaiborne', 'draculadeadandlovingit',
       'sheworeayellowribbon', 'sandsofiwojima', 'tokillamockingbird',
       'sweetdreams', 'roostercogburn', 'forricherorpoorer',
       'thebreakfastclub', '1941', 'thechangeling', 'oldyeller',
       'prettywoman', 'enchanted', 'goodwillhunting',
       'oneflewoverthecuckoosnest', 'contact', 'seven', 'bladerunner',
       'ladyhawke', 'northbynorthwest', 'asummerplace', 'victorvictoria',
       'titanic', 'thefirstwivesclub', 'inharmsway',
       'indianajonesandthetempleofdoom', 'roadhouse', 'poltergeist',
       'fiddlerontheroof', 'chittychittybangbang', 'mrmom',
       'thesilenceofthelambs', 'thebestyearsofourlives',
       'howthewestwaswon',
       'drstrangeloveorhowilearnedtostopworryingandlovethebomb',
       'sensesensibility', 'themuppetchristmascarol', 'ulysses',
       'warlock', 'oklahoma', 'slingbl

In [20]:
df_merge.title_normalize.value_counts()

red                                    258
theloneranger                          111
achristmascarol                        105
robinhood                               96
safehouse                               85
sherlockholmes                          72
journeytothecenteroftheearth            72
thedaytheearthstoodstill                69
prometheus                              66
therevenant                             66
unknown                                 64
inharmsway                              63
sabotage                                57
totalrecall                             56
red2                                    55
anastasia                               55
argo                                    54
independenceday                         54
thelordoftheringsthereturnoftheking     52
guardiansofthegalaxy                    52
nowyouseeme                             51
taken                                   50
underworld                              50
payback    